In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
##Read The Data 
traning=spark.read.csv('TEST2.csv',header=True,inferSchema=True)

In [3]:
traning.show()

+--------+---+--------+------+
|    Name|Age|exprince|Salary|
+--------+---+--------+------+
|Shreyas | 31|      10| 30000|
|   Aishu| 30|       8| 20000|
| Ramesh | 29|       4| 25000|
|    Tuka| 24|       4| 20000|
| Mahesh | 34|       6| 40000|
| Tukaram| 34|      10| 60000|
|    Kaka| 36|       8| 55000|
|    Paul| 24|       3| 20000|
|  Harsha| 21|       1| 15000|
+--------+---+--------+------+



In [4]:
traning.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- exprince: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
traning.columns

['Name', 'Age', 'exprince', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler= VectorAssembler(inputCols=["Age","exprince"],outputCol="Independent Features")

In [8]:
output = featureassembler.transform(traning)
output.show()

+--------+---+--------+------+--------------------+
|    Name|Age|exprince|Salary|Independent Features|
+--------+---+--------+------+--------------------+
|Shreyas | 31|      10| 30000|         [31.0,10.0]|
|   Aishu| 30|       8| 20000|          [30.0,8.0]|
| Ramesh | 29|       4| 25000|          [29.0,4.0]|
|    Tuka| 24|       4| 20000|          [24.0,4.0]|
| Mahesh | 34|       6| 40000|          [34.0,6.0]|
| Tukaram| 34|      10| 60000|         [34.0,10.0]|
|    Kaka| 36|       8| 55000|          [36.0,8.0]|
|    Paul| 24|       3| 20000|          [24.0,3.0]|
|  Harsha| 21|       1| 15000|          [21.0,1.0]|
+--------+---+--------+------+--------------------+



In [10]:
output.columns

['Name', 'Age', 'exprince', 'Salary', 'Independent Features']

In [11]:
    finalized_data = output.select("Independent Features","Salary")

In [12]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 20000|
|          [29.0,4.0]| 25000|
|          [24.0,4.0]| 20000|
|          [34.0,6.0]| 40000|
|         [34.0,10.0]| 60000|
|          [36.0,8.0]| 55000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
##train test Spilt 
train_data,test_data =finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features',labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [15]:
##Regressore Coeffcients
regressor.coefficients

DenseVector([2179.5487, 818.531])

In [16]:
##Reressor Intercept 
regressor.intercept

-36082.57321171403

In [17]:
##Predictions 
pred_result = regressor.evaluate(test_data)

In [18]:
pred_result.predictions.show()

C:\Users\shreyas.awati\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18682.189150264036|
|          [29.0,4.0]| 25000|30398.463754200697|
|         [31.0,10.0]| 30000| 39668.74699951992|
|          [36.0,8.0]| 55000| 48929.42870859342|
+--------------------+------+------------------+



In [19]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(5613.898223715791, 40304135.17146893)